In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore

In [3]:
data = pd.read_csv('../data/togo-dapaong_qc.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525600 entries, 0 to 525599
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Timestamp      525600 non-null  object 
 1   GHI            525600 non-null  float64
 2   DNI            525600 non-null  float64
 3   DHI            525600 non-null  float64
 4   ModA           525600 non-null  float64
 5   ModB           525600 non-null  float64
 6   Tamb           525600 non-null  float64
 7   RH             525600 non-null  float64
 8   WS             525600 non-null  float64
 9   WSgust         525600 non-null  float64
 10  WSstdev        525600 non-null  float64
 11  WD             525600 non-null  float64
 12  WDstdev        525600 non-null  float64
 13  BP             525600 non-null  int64  
 14  Cleaning       525600 non-null  int64  
 15  Precipitation  525600 non-null  float64
 16  TModA          525600 non-null  float64
 17  TModB          525600 non-nul

In [5]:
data.describe()

,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
count,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,525600.000000,0.0
mean,230.555040,151.258469,116.444352,226.144375,219.568588,27.751788,55.013160,2.368093,3.229490,0.557740,161.741845,10.559568,975.915242,0.000535,0.001382,32.444403,33.543330,NaN
std,322.532347,250.956962,156.520714,317.346938,307.932510,4.758023,28.778732,1.462668,1.882565,0.268923,91.877217,5.915490,2.153977,0.023116,0.026350,10.998334,12.769277,NaN
min,-12.700000,0.000000,0.000000,0.000000,0.000000,14.900000,3.300000,0.000000,0.000000,0.000000,0.000000,0.000000,968.000000,0.000000,0.000000,13.100000,13.100000,NaN
25%,-2.200000,0.000000,0.000000,0.000000,0.000000,24.200000,26.500000,1.400000,1.900000,0.400000,74.800000,6.900000,975.000000,0.000000,0.000000,23.900000,23.600000,NaN
50%,2.100000,0.000000,2.500000,4.400000,4.300000,27.200000,59.300000,2.200000,2.900000,0.500000,199.100000,10.800000,976.000000,0.000000,0.000000,28.400000,28.400000,NaN
75%,442.400000,246.400000,215.700000,422.525000,411.000000,31.100000,80.800000,3.200000,4.400000,0.700000,233.500000,14.100000,977.000000,0.000000,0.000000,40.600000,43.000000,NaN
max,1424.000000,1004.500000,805.700000,1380.000000,1367.000000,41.400000,99.800000,16.100000,23.100000,4.700000,360.000000,86.900000,983.000000,1.000000,2.300000,70.400000,94.600000,NaN


In [6]:
def find_columns_with_missing_value(df:pd.DataFrame, threshold=0.05)->list:
    null_columns = df.isnull().sum()
    total_row = len(df)
    null_percentage = (null_columns/total_row)*100;
    missing_columns = df.columns[null_percentage > threshold]
    print('columns above the threshold')
    return missing_columns.to_list()

In [7]:
df_copy = data.copy()

In [8]:
df_copy['Timestamp'] = pd.to_datetime(data['Timestamp'])

In [ ]:
target_cols = [ 'GHI', 'DNI', 'DHI', 'ModA', 'ModB','WS', 'WSgust']

In [9]:
def find_and_replace_outliers_with_median(df, cols, iqr_multiplier=1.5):
    """
    Finds outliers in specified columns using the IQR method and replaces them
    with the median of their respective columns.

    Args:
        df (pd.DataFrame): The input pandas DataFrame.
        cols (list): A list of column names to check for outliers and replace.
        iqr_multiplier (float, optional): The multiplier for the IQR to define
                                           the outlier bounds. Defaults to 1.5
                                           (standard for box plots).

    Returns:
        pd.DataFrame: A new DataFrame with outliers replaced by the median.
                      Returns a copy, the original DataFrame is not modified.
    """
    df_cleaned = df.copy() # Create a copy to avoid modifying the original DataFrame

    print(f"Processing columns: {cols}")

    for col in cols:
        if col not in df.columns:
            print(f"Warning: Column '{col}' not found in DataFrame. Skipping.")
            continue

        # Ensure the column is numeric, as IQR and median are for numerical data
        if not pd.api.types.is_numeric_dtype(df_cleaned[col]):
             print(f"Warning: Column '{col}' is not numeric. Skipping outlier detection/replacement.")
             continue

        # Calculate Q1, Q3, and IQR for the current column
        Q1 = df_cleaned[col].quantile(0.25)
        Q3 = df_cleaned[col].quantile(0.75)
        IQR = Q3 - Q1

        # Define the lower and upper bounds for outlier detection
        lower_bound = Q1 - iqr_multiplier * IQR
        upper_bound = Q3 + iqr_multiplier * IQR

        # Identify outliers in the current column
        # Create a boolean mask: True for values outside the bounds
        outlier_mask = (df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)

        # Get the indices of the outliers for this specific column
        outlier_indices_col = df_cleaned.index[outlier_mask]

        # Check if any outliers were found in this column
        if len(outlier_indices_col) == 0:
            print(f"No outliers found in column '{col}' using IQR multiplier {iqr_multiplier}.")
            continue

        print(f"Found {len(outlier_indices_col)} outliers in column '{col}'.")
        # print(f"Outlier indices in '{col}': {outlier_indices_col.tolist()}") # Uncomment to see indices

        # Calculate the median for the current column
        median_value = df_cleaned[col].median()

        print(f"Median value for '{col}' (used for replacement): {median_value}")

        # Replace the outlier values with the calculated median
        df_cleaned.loc[outlier_indices_col, col] = median_value

        print(f"Outliers in column '{col}' replaced with median.")

    return df_cleaned

In [ ]:
def clean_data(df:pd.DataFrame,cols:list)-> pd.DataFrame:
    clean_data = df.drop(columns=['Comments']).ffill()
    for col in cols:
        clean_data[col]=clean_data[col].clip(lower=0)
    clean_data['RH'].clip(0, 100)
    clean_data = find_and_replace_outliers_with_median(clean_data, cols + ['Tamb'])
    
    return clean_data.reset_index(drop=True)

In [ ]:
def data_quality_report(df: pd.DataFrame) -> pd.DataFrame:
    """Generate comprehensive data quality report"""
    report = pd.DataFrame({
        'Missing Values': df.isna().sum(),
        'Zero Values': (df == 0).sum(),
        'Negative Values': (df.select_dtypes(include=np.number) < 0).sum()
    })

    # Value range checks
    ranges = {
        'GHI': (0, 1500),
        'RH': (0, 100),
        'Tamb': (-20, 60)
    }
    for col, (min_val, max_val) in ranges.items():
        report.loc[col, 'Out of Range'] = ((df[col] < min_val) | (df[col] > max_val)).sum()

    return report
